In [5]:
pip install request

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


In [6]:
pip install beautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [27]:
import time
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from selenium.webdriver import ActionChains

driver = webdriver.Chrome('C:/Users/KSY/Desktop/chromedriver.exe')
driver.get("https://www.idus.com") #tab[0]
driver.execute_script('window.open("about:blank", "_blank");') #창 1개더 열어놓기 tab[1]
tabs = driver.window_handles
time.sleep(3)

def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

with open ("ourdus_product.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)
    time.sleep(3)
    driver.switch_to_window(tabs[0])
    scroll()
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    myWriter.writerow(["카테고리", "작품이름","배너 사진 URL","작가이름", "할인율",'가격','상세페이지 URL','적립금','배송비','배송시작','수량,','준비물','작품정보','후기'])
       
    classlist = soup.findAll('li',{'class':'ui_grid__item'}) #모든 작품 목록
    count = 0
    
    data=[] 
    cnt=0
    #클래스 한개씩 작업
    for row in classlist: 
        if cnt>10:
            break
        cnt+=1
        #img_URL얻으려고 전처리과정
        img = row.find('a',{'class':'ui_card__img'})
        img = img["style"]
        start=img.find("\'")
        end = img.find("\'",start+1)
        img_URL = img[start+1:end]

        #img_URL로 로컬 저장소에 사진 저장
        with urlopen(img_URL) as f:
            count +=1
            with open('./Ourdus/product_img/'+ str(count)+ '.jpg',"wb") as h:
                img=f.read()
                h.write(img)
        
        
#         label = row.find('div',{'class':'label'}).text #카테고리
        writername=row.find('a',{'class':'ui_card__label'}).text
        productname = row.find('a',{'class':'ui_card__title'}).text 

        
#         # 할인률 없는것들도 있어서 try~catch 사용
#         try:
#             discount = row.find('span',{'class':'sale-rate'}).text.replace('%','')
#             discount=int(discount)
#         except:
#             discount=0
            
#         #가격 전처리 (/월 가격임)
#         price = row.find('span',{'class':'price'}).text.replace('\n','').replace(',','')
#         price = int(price)
        
#         data.append([label,name,img_URL,discount,price])
         
        # 현재 클래스의 상세 페이지 URL
        info_URL = row.find('a',{'class':'ui_card__img'})
        info_URL = info_URL["href"]
        info_URL = "https://www.idus.com/" + str(info_URL)
        
        
        driver.switch_to_window(tabs[1]) #두번째 탭으로 이동
        driver.get(info_URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        try:
            discount = row.find({'em':'data-v-2f102c5f'}).text
            discount=int(discount)
        except:
            discount=0
        price_info = row.find('div',{'class':'price_tag-detail'})
        detail_info = row.find('div',{'class':'product-define-info'})
#         point = row.find('span',{'class':'bold-txt'}).text.replace('P','')
#         point = int(point)
#         purchase = row.find({'div':'data-v-ea27ca2a'},{'style':'margin-left: 3px'}).text.replace(')','').replace('(','')
#         purchase = int(purchase)
#         delivery = row.find({'span':'data-v-9e98616a'}).text.replace("원","").replace(",","")
#         delivery=int(delivery)
#         stock = row.find({'span':'data-v-9e98616a'}).text
        
        #클래스 정보 소개
        article = soup.findAll('p',{'class':'para'})
        article_info=[]
        for i in article:
            article_info.append(i.text)
        
        #클래스 댓글
#         comment = soup.findAll('div',{'class':'area-txt'})
#         comment_info=[]
#         for i in comment:
#             try:
#                 user_name=i.find({'span':'data-v-698f8bea'}).text
#                 user_comment=i.find('p',{'class':'txt'}).text
#                 comment_info.append([user_name,user_comment])   # [작성자 이름, 코멘트 내용] 이 한묶음
#             except:
#                 continue

#         tmp = data[count-1]+[total_price,info_URL,start_date,duration,difficulty,material,article_info,comment_info]
        
#         myWriter.writerow(tmp)
#         print(tmp)
        print(writername, productname,info_URL,discount,price_info,detail_info,article)
        driver.switch_to_window(tabs[0])
        
driver.close()
    
 

<ipython-input-27-9748d7f83763>:35: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[0])
<ipython-input-27-9748d7f83763>:91: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[1]) #두번째 탭으로 이동


선물정육점(고기셰프 윤주민과청년들) 설날선물세트🎁1+등급 한우🎁감사한우세트 1 https://www.idus.com//w/product/87ef03bf-97ff-42e6-88b5-f72dba271d8a 0 None None [<p class="para" style="word-break: break-all">  🟥 배송 공지 드려요~♡ 🟥<br/>
<br/>
♦️지정일 배송 가능 👉 주문 주실 때 요청사항에 기재 ♦️<br/>
(지정일 없을 시 오전 7시까지 주문 건 당일 출고)<br/>
<br/>
<br/>
🎁선물정육점🎁 한우는요~<br/>
<br/>
인증된 경매기관에서 특별히 엄선된 한우만을 공급받아 <br/>
저희 매장에서 직접 가공 및 포장을 하고 있어요~^^<br/>
<br/>
✔등급, 육색, 마블링, 육질✔ 등<br/>
까다롭고 엄격한 기준을 적용하여 <br/>
기준이상 품질의 한우만 사용하고 있습니다. <br/>
<br/>
한우의 맛을 좌우하는 가장 큰 차이는 <br/>
🌟등급과 숙성기술🌟이에요. <br/>
<br/>
저희는 ✔1+등급이상✔ 한우만 취급하며<br/>
자체 숙성 기술로 15일 이상 숙성된 한우만<br/>
보내드리고 있어요~^^<br/>
<br/>
<br/>
⛔⛔⛔ 웻에이징 숙성한우 ⛔⛔⛔<br/>
<br/>
웻에이징 방식으로 숙성된 한우는 수분이 충분하여 <br/>
부드럽고 진한 맛을 지니며 <br/>
특유의 깊은 풍미가 살아있습니다👍<br/>
<br/>
<br/>
<br/>
⛔⛔  구성 안내 ⛔⛔<br/>
<br/>
선물정육점  🎁등심1호 선물세트🎁<br/>
<br/>
🐴숙성등심400g + 숙성국거리400g + 숙성불고기400g(냉동)🐴<br/>
<br/>
✔1+등급 한우로만 구성!! 총 1200g✔<br/>
<br/>
(넉넉하게 드시라고 부족함 없이 보내드려요~)<br/>
<br/>
🎈보관 기간 및 방법 :  수령 후 7일 이내 냉장 보관🎈<br/>
<br/>
<br/>
<br/>
⛔⛔ 배송 안내 ⛔⛔<br/>
<br/>
☝️  M

<ipython-input-27-9748d7f83763>:132: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[0])


전남장흥 무태장어 ‼️설날이벤트‼️쫀득쫀득 탱탱 전남장흥 "무태장어" https://www.idus.com//w/product/5ea1614f-fd34-4f5c-b2c9-4edac13a296e 0 None None [<p class="para" style="word-break: break-all">‼️설날이벤트‼️<br/>
(1월21일~ 2월6일)<br/>
<br/>
🔴설날전 마지막 출고🔴<br/>
🔴🔴2월7일(일)🔴🔴<br/>
🔴설날 조기품절 우려가 있으니🔴<br/>
🙏예약권장 해드려요🙏<br/>
<br/>
보양식의 끝판왕인<br/>
장어👍 설날이벤트로 <br/>
할인가격 + kg당 장어즙1팩씩 증정🤗<br/>
<br/>
만남이 어려운 시기<br/>
이벤트를 통해<br/>
소중한분들께 건강한 선물을💜<br/>
소중한분들과 면역력증가에<br/>
좋은 식사를💜<br/>
<br/>
❗배송관련❗<br/>
월~금 오후3시이전 주문건<br/>
 당일출고 가능합니다.<br/>
(예약주문가능)<br/>
<br/>
🔴제주도,도서산간🔴<br/>
발송마감❌❌<br/>
<br/>
🧊포장구성🧊<br/>
진공포장+아이스팩+보냉팩+아이스박스<br/>
총4중 신선포장<br/>
수령후 5~7일까지 냉장보관 가능하세요😄<br/>
<br/>
❗상품구성❗<br/>
1kg (손질후 실중량 700~750g)<br/>
     평균2~3마리 (2~3인분)<br/>
kg당 생강채+매,순소스 각각1개씩+소금<br/>
<br/>
❗맛있게 드시는법❗<br/>
(설명서동봉) <br/>
⭐앞, 뒤, 자르고 옆면까지⭐<br/>
⭐노릇하게 구워질수록⭐<br/>
잡내와 잔가시가 느껴지지 않습니다<br/>
<br/>
🌱"맑은물 푸른숲" 전라남도 장흥에서<br/>
아버지와 함께 귀어하여 8년째<br/>
민물장어 양식장을 운영하고 있는<br/>
"다보수산" 대표<br/>
청년어업인 김창주 입니다.<br/>
<br/>
✔첫째 (식감과 맛이 일품인 명품장어👍)<br/>
<br/>
“무태장어” 는 민물장어의 한종류에 속한 <

일랑(YLANG) 일랑2013🌸 명절특가 사전예약🌸 더치앰플10 https://www.idus.com//w/product/8df62619-d2c5-4c75-9253-cc8d128b73a2 0 None None [<p class="para" style="word-break: break-all"> ● 주문전 필수 안내 ●<br/>
<br/>
현재 예약발송 진행중입니다.<br/>
기간 : 2021.01.15. ~ 2021.02.03.<br/>
발송 : 2021.02.03. ~ 순차발송해드립니다.<br/>
<br/>
당일 구매는 2021.02.05.일 15시까지 진행되며 <br/>
별도의 할인 진행됩니다.<br/>
2021.02.05.일 15시이후 부터의 주문건은 <br/>
15일부터 순차적으로 발송됩니다.<br/>
<br/>
설 연휴 기간에 문의에 대한 답변이 원활하지 않을수 있음을 양해부탁드립니다.<br/>
<br/>
<br/>
 ●   제 품 소 개   ●<br/>
<br/>
용량 : 20ml × 10ea<br/>
구성 : 20ml 10가지 맛 각1ea<br/>
박스 규격 : 275 × 95 × 25<br/>
쇼핑백 규격 : 290 × 150 × 60<br/>
<br/>
<br/>
<br/>
●  배 송 정 보  ●<br/>
<br/>
월~금요일 주문시 :  당일발송,익일배송<br/>
   토~일요일 주문시 : 일요일발송,익일배송<br/>
(단,일요일15시 이후 주문시 : 월요일발송,익일배송)<br/>
<br/>
<br/>
<br/>
<br/>
● 테 이 스 팅 노 트 ●<br/>
<br/>
콜롬비아  -  초콜릿, 헤이즐넛<br/>
        케냐  - 코코아, 아몬드, 오렌지<br/>
엘살바도르  -  갈색설탕, 팝콘, 캐슈넛<br/>
에티오피아  -  초콜릿, 와인<br/>
브라질 -  흑설탕, 군밤, 땅콩버터<br/>
인도네시아 - 갈색설탕, 초콜릿, 자두<br/>
과테말라 -  밀크초콜릿, 호두<br/>
탄자니아 -  초콜릿, 홍자, 아몬드<br/>